In [1]:
import time
import pandas as pd
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
from datetime import datetime

# Tarayıcı ayarları
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)


service = Service(ChromeDriverManager().install())

# WebDriver'ı başlatma
driver = webdriver.Chrome(service=service, options=options)


driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")


data = []

# Temizleme fonksiyonları
def clean_price(text):
    if not text:
        return 'N/A'

    num_text = re.sub(r'[^\d]', '', text)
    return int(num_text) if num_text else 'N/A'

def clean_room(text):
    if not text:
        return 'N/A'
    return text.replace('\n', '').replace(' ', '').strip()

def clean_size(text):
    if not text:
        return 'N/A'
    match = re.search(r'(\d+)', text)
    return int(match.group(1)) if match else 'N/A'

def clean_age(text):
    if not text:
        return 'N/A'
    text = text.strip()
    match = re.search(r'(\d+)', text)
    return int(match.group(1)) if match else text

def clean_floor(text):
    if not text:
        return 'N/A'

    text = text.strip()
    return text.replace('Floor', '').strip()


toplam_sayfa = 100 # İsterseniz değiştirebilirsiniz


bugün = datetime.now().strftime('%Y%m%d')


total_listings = 0

try:
    for page in range(1, toplam_sayfa + 1):
        if page == 1:
            url = 'https://www.hepsiemlak.com/en/ankara-kiralik/daire'
        else:
            url = f'https://www.hepsiemlak.com/en/ankara-kiralik/daire?page={page}'

        print(f"Sayfa {page}/{toplam_sayfa} yükleniyor: {url}")
        driver.get(url)

        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.listView"))
            )
        except:
            print("Sayfa yüklenemedi, tekrar deneniyor...")
            driver.refresh()
            time.sleep(5)
            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.listView"))
                )
            except:
                print("Sayfa yüklenemiyor, sonraki sayfaya geçiliyor.")
                continue

        try:
            WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
            ).click()
            print("Çerezler kabul edildi")
        except:
            pass

        for _ in range(5):
            driver.execute_script("window.scrollBy(0, window.innerHeight * 0.8);")
            time.sleep(1)

        html = driver.page_source


        soup = BeautifulSoup(html, 'html.parser')


        listings = soup.find_all('div', class_='list-view-content')

        if not listings:

            listings = soup.find_all('div', class_='listing-item')

        page_listings = 0


        for listing in listings:
            try:
                # Fiyat
                price_tag = listing.find('span', class_='list-view-price')
                price_text = price_tag.get_text(strip=True) if price_tag else 'N/A'
                price = clean_price(price_text)

                # Tarih
                date_tag = listing.find('span', class_='list-view-date')
                date = date_tag.get_text(strip=True) if date_tag else 'N/A'

                # Başlık
                title_tag = listing.find('h3')
                title = title_tag.get_text(strip=True) if title_tag else 'N/A'

                # Konum
                location_tag = listing.find('span', class_='list-view-location')
                location = location_tag.get_text(strip=True) if location_tag else 'N/A'

                # Oda sayısı
                room_tag = listing.find('span', class_='houseRoomCount')
                room_text = room_tag.get_text(strip=True) if room_tag else 'N/A'
                room = clean_room(room_text)

                # Metrekare
                size_tag = listing.find('span', class_='squareMeter')
                size_text = size_tag.get_text(strip=True) if size_tag else 'N/A'
                size = clean_size(size_text)

                # Bina yaşı
                age_tag = listing.find('span', class_='buildingAge')
                age_text = age_tag.get_text(strip=True) if age_tag else 'N/A'
                age = clean_age(age_text)

                # Kat
                floor_tag = listing.find('span', class_='floortype')
                floor_text = floor_tag.get_text(strip=True) if floor_tag else 'N/A'
                floor = clean_floor(floor_text)

                # İlan linki - a etiketini bul
                link_tag = listing.find('a', class_='card-link')
                link = link_tag['href'] if link_tag and 'href' in link_tag.attrs else 'N/A'
                if link != 'N/A' and not link.startswith('http'):
                    link = f"https://www.hepsiemlak.com{link}"


                data.append({
                    'Title': title,
                    'Price (TL)': price,
                    'Date': date,
                    'Location': location,
                    'Room': room,
                    'Size (m²)': size,
                    'Building Age': age,
                    'Floor': floor,
                    'Link': link,
                    'Page': page
                })

                page_listings += 1

            except Exception as e:
                print(f"İlan verisi çekilirken hata: {e}")

        total_listings += page_listings
        print(f"Sayfa {page}: {page_listings} ilan çekildi. Toplam: {total_listings}")


        if page_listings == 0:
            print("Bu sayfada ilan bulunamadı. Veri çekme sonlandırılıyor.")
            break


        if page % 10 == 0 or page == toplam_sayfa:
            df_temp = pd.DataFrame(data)
            ara_dosya = f"ankara_kiralik_ara_sayfa_{page}_{bugün}.xlsx"
            df_temp.to_excel(ara_dosya, index=False)
            print(f"Ara sonuçlar '{ara_dosya}' dosyasına kaydedildi.")

        # Bot tespitini engellemek için rastgele bekleme süresi
        wait_time = random.uniform(3, 6)
        print(f"{wait_time:.1f} saniye bekleniyor...")
        time.sleep(wait_time)

except Exception as e:
    print(f"Genel hata: {e}")

finally:

    driver.quit()

    if data:

        df = pd.DataFrame(data)


        numeric_cols = ['Price (TL)', 'Size (m²)', 'Building Age']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')


        excel_dosya = f"ankara_kiralik_daireler_{bugün}.xlsx"
        csv_dosya = f"ankara_kiralik_daireler_{bugün}.csv"

        df.to_excel(excel_dosya, index=False)
        df.to_csv(csv_dosya, index=False, encoding='utf-8-sig')

        print(f"\nVeriler başarıyla '{excel_dosya}' ve '{csv_dosya}' dosyalarına kaydedildi.")
    else:
        print("Hiç veri çekilemedi!")

Sayfa 1/1 yükleniyor: https://www.hepsiemlak.com/en/ankara-kiralik/daire
Sayfa 1: 24 ilan çekildi. Toplam: 24
Ara sonuçlar 'ankara_kiralik_ara_sayfa_1_20250323.xlsx' dosyasına kaydedildi.
3.0 saniye bekleniyor...

Veriler başarıyla 'ankara_kiralik_daireler_20250323.xlsx' ve 'ankara_kiralik_daireler_20250323.csv' dosyalarına kaydedildi.
